<a href="https://colab.research.google.com/github/LeandroCoelhos/estudos_datascience/blob/main/regressao_interpretabilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Carregando o conjunto de dados da Califórnia
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Dividindo o conjunto de dados em treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Separando variáveis independentes (X) e a variável dependente (y)
X_train = train_df.drop('target', axis=1)
y_train = train_df['target']

X_test = test_df.drop('target', axis=1)
y_test = test_df['target']

# Adicionando uma constante para o termo independente
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Criando e treinando o modelo de regressão linear
modelo_reg = sm.OLS(y_train, X_train).fit()

# Realizando previsões no conjunto de teste
y_pred = modelo_reg.predict(X_test)

# Calculando as métricas
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Imprimindo as métricas
print(f"R2 Score: {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Imprimindo os coeficientes do modelo
print("\nCoeficientes do Modelo:")
print(modelo_reg.params)

# Identificando as três variáveis mais impactantes
top_variables = modelo_reg.params.abs().sort_values(ascending=False).head(3)
print("\nAs três variáveis mais impactantes:")
print(top_variables)


R2 Score: 0.5758
Mean Squared Error (MSE): 0.5559
Mean Absolute Error (MAE): 0.5332

Coeficientes do Modelo:
const        -37.023278
MedInc         0.448675
HouseAge       0.009724
AveRooms      -0.123323
AveBedrms      0.783145
Population    -0.000002
AveOccup      -0.003526
Latitude      -0.419792
Longitude     -0.433708
dtype: float64

As três variáveis mais impactantes:
const        37.023278
AveBedrms     0.783145
MedInc        0.448675
dtype: float64


In [14]:
import statsmodels.api as sm
import pandas as pd

# Supondo que você já treinou um modelo de regressão linear chamado modelo_reg
# e tem um conjunto de teste chamado X_test

# Adicionando uma constante à matriz de recursos (intercepto)
X_test_com_const = sm.add_constant(X_test)

# Obtendo as previsões do modelo
predictions = modelo_reg.get_prediction(X_test_com_const)

# Inicializando um DataFrame para armazenar os resultados
resultados_df = pd.DataFrame(index=X_test.index)

# Iterando sobre cada previsão individual
for i, (idx, row) in enumerate(predictions.summary_frame().iterrows()):
    # Obtendo os coeficientes para a previsão atual
    coeficientes_previsao = row[:-1]  # Excluindo o intercepto

    # Extraindo os três coeficientes mais influentes
    coeficientes_influentes = coeficientes_previsao.abs().nlargest(3).index

    # Salvando os três coeficientes mais influentes nas colunas correspondentes
    resultados_df.at[idx, 'mais_influente_1'] = coeficientes_influentes[0]
    resultados_df.at[idx, 'mais_influente_2'] = coeficientes_influentes[1]
    resultados_df.at[idx, 'mais_influente_3'] = coeficientes_influentes[2]

# Exibindo o DataFrame resultante
print(resultados_df)

      mais_influente_1 mais_influente_2 mais_influente_3
20046    mean_ci_upper             mean    mean_ci_lower
3024     mean_ci_upper             mean    mean_ci_lower
15663    mean_ci_upper             mean    mean_ci_lower
20484    mean_ci_upper             mean    mean_ci_lower
9814     mean_ci_upper             mean    mean_ci_lower
...                ...              ...              ...
15362    mean_ci_upper             mean    mean_ci_lower
16623    mean_ci_upper             mean    mean_ci_lower
18086    mean_ci_upper             mean    mean_ci_lower
2144     mean_ci_upper             mean    mean_ci_lower
3665     mean_ci_upper             mean    mean_ci_lower

[4128 rows x 3 columns]


In [18]:
# Criando uma lista com todas as variáveis mais influentes
variaveis_influentes = resultados_df['mais_influente_1'].tolist() + resultados_df['mais_influente_2'].tolist() + resultados_df['mais_influente_3'].tolist()

# Contando a ocorrência de cada variável
contagem_variaveis = pd.Series(variaveis_influentes).value_counts()

# Obtendo os top 3 valores mais comuns
top3_valores_mais_comuns = contagem_variaveis.head(3)

# Exibindo os top 3 valores mais comuns
print(top3_valores_mais_comuns)

mean             4125
mean_ci_upper    4115
mean_ci_lower    3930
dtype: int64
